In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/processed_salaries.csv")
df.head()

Mounted at /content/drive


,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_size,salary_binned
0,2024,EN,CT,Other,88888,Other,0,L,0
1,2024,EX,FT,Other,106666,Other,100,L,0
2,2024,EX,FT,Other,106666,Other,100,L,0
3,2024,SE,FT,Data Engineer,130500,US,0,M,0
4,2024,SE,FT,Data Engineer,96000,US,0,M,0


In [2]:
#Convert categorical data to numeric with 'pd.get_dummies'
df = pd.get_dummies(df, columns=['experience_level',  'job_title',  'employee_residence', 'company_size'])
df.head()

,work_year,employment_type,salary_in_usd,remote_ratio,salary_binned,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,job_title_Analytics Engineer,...,job_title_Data Scientist,job_title_Machine Learning Engineer,job_title_Other,job_title_Research Engineer,job_title_Research Scientist,employee_residence_Other,employee_residence_US,company_size_L,company_size_M,company_size_S
0,2024,CT,88888,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
1,2024,FT,106666,100,0,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,0
2,2024,FT,106666,100,0,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,0
3,2024,FT,130500,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
4,2024,FT,96000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0


In [3]:
# Split our preprocessed data into our features and target arrays. Here we make salary_in_usd as our target array as that is what we would like predicted
y = df['salary_binned'].values
X = df.drop(columns=['salary_in_usd', 'salary_binned', 'employment_type']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Convert target variable to one-hot encoded format
num_bins = 5
num_classes = 5  # Number of classes for one-hot encoding
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)

# Compile, Train and Evaluate the Model

In [12]:

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_bins, activation='softmax')  # Output layer with num_bins units and softmax activation
])

# Check the structure of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               2944      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 11525 (45.02 KB)
Trainable params: 11525 (45.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model with appropriate loss function for multi-class classification
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
# Train the model
model.fit(X_train_scaled, y_train_encoded, epochs=100, verbose=1)

Epoch 1/100
267/267 [==============================] - 2s 2ms/step - loss: 0.9546 - accuracy: 0.5871
Epoch 2/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8996 - accuracy: 0.6023
Epoch 3/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8920 - accuracy: 0.6002
Epoch 4/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8846 - accuracy: 0.6048
Epoch 5/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8826 - accuracy: 0.6043
Epoch 6/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8816 - accuracy: 0.6074
Epoch 7/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8794 - accuracy: 0.6055
Epoch 8/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8763 - accuracy: 0.6057
Epoch 9/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8736 - accuracy: 0.6078
Epoch 10/100
267/267 [==============================] - 1s 2ms/step - loss: 0.8732 - accura

In [ ]:
# Evaluate the model on the test set
model_loss, model_accuracy = model.evaluate(X_test_scaled, tf.keras.utils.to_categorical(y_test, num_classes=num_classes), verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

91/91 - 0s - loss: 1.2341 - accuracy: 0.4277 - 255ms/epoch - 3ms/step
Loss: 1.2341376543045044, Accuracy: 0.42772963643074036
